In [66]:
from py2neo import Graph, NodeMatcher, RelationshipMatcher
import pandas as pd
from datetime import date

In [67]:
# connect to the graph -> it should be running already
graph = Graph(scheme="bolt", host="localhost", port=7687, auth=('neo4j', '12345'))

# clean up the database and PERMANENTLY delete all nodes and relationships
#graph.delete_all()
# clean up only a elements satisfying a specific MATCH
#query = "MATCH (n) WHERE n.name=.. delete n"
#graph.run(query)

##  ----------------  ##
# development milestone: add all functions to a class that holds the graph as a member
##  ----------------  ##

In [123]:
# utility functions ready to be used

class neo2BFMPolymer:
    """Python class to work on a neo4j graph data base with bfm polymer simulation data.
    
    Attributes:
        graph (py2neo.Graph): instance of the graph to access the neo4j database
        nodeMatcher (py2neo.NodeMatcher): matcher for the graph to evaluate nodes according to certain criteria.
        nodeMatcher (py2neo.RelationshipMatcher): matcher for the graph to evaluate relationships according to certain criteria.
    """
    
    def __init__(self, graph):
        """Constructor of the neo2BFMPolmyer class
        
        Parameters:
            graph (py2neo.Graph): instance of the graph to access the neo4j database
        """
        self.graph       = graph
        self.nodeMatcher = NodeMatcher(self.graph)
        self.relaMatcher = RelationshipMatcher(self.graph)
    
    
    def addNodeGeneral(self, nodeTypeName, nodeName):
        """Utility function adding a new node if it does not already exist.
        
        Paramters:
            nodeType (str): name of the node type
            nodeName (str): name property of the new node
            
        Returns:
            exit code (bool): True if node was added, False if node already exists
        """
        elementExists = self.graph.run("MATCH (elem:{}) WHERE elem.name=\"{}\" return elem".format(nodeTypeName,nodeName)).data()
        if (len(elementExists) > 0):
            print("WARNING: node of type {} with name {} already exists!".format(nodeTypeName,nodeName))
            return False
        else:
            query = "CREATE (pl:{} {{ name: \'{}\', createdOn: \'{}\' }} )".format(nodeTypeName, nodeName, date.today())
            self.graph.run(query)
            return True
    
    
    def addNewPolymer(self, polymerName):
        """Adding a new Polymer node to the database.
        
        This function adds a node of type Polymer with the given name and a date tag to the database.
        If the database already contains a Polymer node with this name property, a warnig is printed
        and no create query is called to avoid node doubling.
        
        Paramters:
            polymerName (str): name property of the new Polymer node
            
        Returns:
            exit code (bool): True if node was added, False if node already exists
        """
        nodeTypeName = "Polymer"
        nodePropertyName = polymerName
        
        return self.addNodeGeneral(nodeTypeName, nodePropertyName)
    
    
    def addNewSimulationType(self, simulationTypeName):
        """Adding a new Simulation node to the database.
        
        A simulation node represents a certain simulation setup such as "linear polymer melt" or "single star polymer".
        This function adds a node of type Simulations with the given name and a date tag to the database.
        If the database already contains a Simulation node with this name property, a warnig is printed
        and no create query is called to avoid node doubling.
        
        Paramters:
            simulationTypeName (str): name property of the new Simulation node
            
        Returns:
            exit code (bool): True if node was added, False if node already exists
        """
        nodeTypeName = "Simulation"
        nodePropertyName = simulationTypeName
        
        return self.addNodeGeneral(nodeTypeName, nodePropertyName)
    
    
    def connectSimulationToPolymer(self,polymerName,simulationName):
        """Connecting a node of type Polymer with a node of type Simulation by a CONTAINS
        
        A connection of type CONTAINS is added between a Polymer node and a Simualtion node,
        heading in the direction of the polymer.
        The connection is only created if the nodes exist and the connection did not already exist.
        In this case, a warning is printed.
        
        Parameters:
            polymerName (str): name of the Polymer node
            simulationName (str): name of the Simulation node
            
        Returns:
            exit code (bool): True if connection was added, False if connection already exists or nodes do not exist
        """
        nodeType1 = "Polymer"
        nodeName1 = polymerName
        nodeType2 = "Simulation"
        nodeName2 = simulationName
        connectionType = "CONTAINS"
        
        elementExists1 = self.graph.evaluate("MATCH (elem:{}) WHERE elem.name=\"{}\" return elem.name".format(nodeType1,nodeName1))
        elementExists2 = self.graph.evaluate("MATCH (elem:{}) WHERE elem.name=\"{}\" return elem.name".format(nodeType2,nodeName2))
        
        if (elementExists1 == nodeName1):
            if (elementExists2 == nodeName2):
                query = "MATCH( ({} {{ name: \'{}\'}}) <- [rel:{}] - ({} {{ name: \'{}\'}}) ) return rel".format(
                    nodeType1,nodeName1,connectionType,nodeType2,nodeName2)
                elementExists = self.graph.evaluate(query)
                if (elementExists == None):
                    query = '''MATCH (p:{}) WHERE p.name=\"{}\"
                               MATCH (s:{}) WHERE s.name=\"{}\"
                               MERGE (s)-[r:{}]->(p)
                    '''.format(nodeType1,nodeName1,nodeType2,nodeName2,connectionType)
                    self.graph.run(query)
                    return True
                
                else:
                    print("WARNING: connection of type {} between {} and {} already exists!".format(connectionType,nodeName1,nodeName2))
                    return False
                
            else:
                print("WARNING: node of type {} with name {} does not exists!".format(nodeType2,nodeName2))
                return False
            
        else:
            print("WARNING: node of type {} with name {} does not exists!".format(nodeType1,nodeName1))
            return False

        

In [126]:
# test the functions
graph.delete_all()
myNeoPolymers = neo2BFMPolymer(graph)
print("add linear chain as polymer: ", myNeoPolymers.addNewPolymer("Linear Chain"))
print("add dendrimers as polymer: ", myNeoPolymers.addNewPolymer("Dendrimer"))
print("add star polymer as polymer: ", myNeoPolymers.addNewPolymer("Star Polymer"))
print("add comb polymer as polymer: ", myNeoPolymers.addNewPolymer("Comb Polymer"))

print("add Linear Polymer Melt as Simulation: ", myNeoPolymers.addNewSimulationType("Linear Polymer Melt"))
print("add Single Linear Polymer as Simulation: ", myNeoPolymers.addNewSimulationType("Single Linear Polymer"))
print("add Single Dendrimer as Simulation: ", myNeoPolymers.addNewSimulationType("Single Dendrimer"))
print("add Mixture Dendrimer and Linear Chains as Simulation: ", myNeoPolymers.addNewSimulationType("Mixture Dendrimer and Linear Chains"))

print("connect polymer and Melt", myNeoPolymers.connectSimulationToPolymer("Linear Chain", "Linear Polymer Melt"))
print("connect polymer and single chain", myNeoPolymers.connectSimulationToPolymer("Linear Chain", "Single Linear Polymer"))
print("connect dendrimers", myNeoPolymers.connectSimulationToPolymer("Dendrimer", "Single Dendrimer"))
print("connect dendrimers", myNeoPolymers.connectSimulationToPolymer("Dendrimer", "Mixture Dendrimer and Linear Chains"))
print("connect dendrimers", myNeoPolymers.connectSimulationToPolymer("Linear Chain", "Mixture Dendrimer and Linear Chains"))


add linear chain as polymer:  True
add dendrimers as polymer:  True
add star polymer as polymer:  True
add comb polymer as polymer:  True
add Linear Polymer Melt as Simulation:  True
add Single Linear Polymer as Simulation:  True
add Single Dendrimer as Simulation:  True
add Mixture Dendrimer and Linear Chains as Simulation:  True
connect polymer and Melt True
connect polymer and single chain True
connect dendrimers True
connect dendrimers True
connect dendrimers True


In [ ]:
## Here some try and error snippets I want to keep for the moment follow:

In [98]:
# Create the generic type 

nodeTypeName = "Simulation"
nodePropertyName = "Comblike Polymer"

elementExists = graph.run("MATCH (elem:{}) WHERE elem.name=\"{}\" return elem".format(nodeTypeName,nodePropertyName)).data()
print(elementExists)
if (len(elementExists) > 0):
    print("WARNING: node of type {} with name {} already exists!".format(nodeTypeName,nodePropertyName))
else:
    query = "CREATE (pl:{} {{ name: \'{}\', createdOn: \'{}\' }} )".format(nodeTypeName, nodePropertyName, date.today())
    print(query)
    graph.run(query)

#query = "MATCH (pD:Polymer) return pD"
#result = graph.run("MATCH (elem:{}) WHERE elem.name=\"{}\" return elem".format(nodeTypeName,nodePropertyName)).data()
#simTypeAlias = Node("Polymer", name = "linear chains")
#CREATE (n:Person { name: 'Andy', title: 'Developer' })
# polymerLinearChain = graph.run("CREATE (pl:Polymer { name: 'Linear Chains'} )")

#polymerLinearChain = graph.run("CREATE (pl:Polymer { name: 'Linear Chains'} )")

[{'elem': (_8:Polymer {createdOn: '2020-06-25', name: 'Comblike Polymer'})}, {'elem': (_9:Polymer {createdOn: '2020-06-25', name: 'Comblike Polymer'})}]


In [23]:
#polymerDendrimer, = graph.run("CREATE (a:Polymer { name: 'Dendrimer'} )")
polymerDendrimer = graph.run("CREATE (pd:Polymer { name: 'Dendrimer'} )")

In [74]:
simDIPS = graph.run("CREATE (a:Simulation {name: 'Mixture of Dendrimers and Linear Polymer Chains'})").data()
print(simDIPS)

[]


In [43]:
#access a single node
query = "MATCH (pD:Polymer) WHERE pD.name='Dendrimer' return pD"
#query = "MATCH (pD:Polymer) return pD"
result = graph.evaluate(query)
# result if of type  py2neo.data.Node
print("print (only!) the first element of MATCH query:\n", result)
print("\naccess properties with [] operator:\t", result["name"])

print (only!) the first element of MATCH query:
 None


TypeError: 'NoneType' object is not subscriptable

In [72]:
query = "MATCH (pD:Polymer) return pD"
result = graph.run(query).data()
print(result)
print(pd.DataFrame(result))

[{'pD': (_0:Polymer {name: 'Linear Chains'})}, {'pD': (_1:Polymer {name: 'Dendrimer'})}]
                          pD
0  {'name': 'Linear Chains'}
1      {'name': 'Dendrimer'}


In [65]:
# Add a new relationship between existing nodes, here with relation type "CONTAINS":

# MATCH (p:Polymer) WHERE p.name="Dendrimer"
# MATCH (s:Simulation) WHERE s.name="Mixture of Dendrimers and Linear Polymer Chains"
# MERGE (s)-[r:CONTAINS]->(p)

query = '''
MATCH (p:Polymer) WHERE p.name="Linear Chains"
MATCH (s:Simulation) WHERE s.name="Mixture of Dendrimers and Linear Polymer Chains"
MERGE (s)-[r:CONTAINS]->(p)
'''
graph.run(query)

query = '''
MATCH (p:Polymer) WHERE p.name="Dendrimer"
MATCH (s:Simulation) WHERE s.name="Mixture of Dendrimers and Linear Polymer Chains"
MERGE (s)-[r:CONTAINS]->(p)
'''
graph.run(query)

py2neo.data.Node

In [53]:
print(res)

In [75]:
query = "MATCH (n:Simulation) WHERE n.name='Mixture of Dendrimers and Linear Polymer Chains' delete n"
graph.run(query)

ClientError: ConstraintValidationFailed: Cannot delete node<2>, because it still has relationships. To delete this node, you must first delete its relationships.